In [1]:
import requests,re,time,numpy as np,pickle,asyncio,concurrent.futures,datetime
# Make this notebook wiiiiiiiiide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

This parser works with a single location, constructing an ITA-style matrix of price vs start/end dates

In [7]:
# Setup relevant variables
ZIP = '60615'
AGENCYBR = 'ET'
#AGENCYID = 'E11509' #oak park
#AGENCYID = 'E11548' #bolingbrook
AGENCYID = 'E115CH' #elmwood
RADIUS = '30'

PICKUP = datetime.date(2017, 11, 7)
DROPOFF = PICKUP+datetime.timedelta(days=7)
FIXEDLENGTH = True
VARYSTART = 6
VARYEND = 2
TIME = '5:00 PM'

In [8]:
%run corefuncs_matrix_debug.ipynb

In [9]:
results = runparser(agencyid = AGENCYID, threads=1)
print(results)

13
(datetime.date(2017, 11, 1), datetime.date(2017, 11, 8))
000 ND | 62 ('enterprise', 'E115CH', 'Dd Elmwood Park') ('14.15', '41.926269', '-87.821333')
(datetime.date(2017, 11, 2), datetime.date(2017, 11, 9))
(datetime.date(2017, 11, 3), datetime.date(2017, 11, 10))
(datetime.date(2017, 11, 4), datetime.date(2017, 11, 11))
NO MATCH
(datetime.date(2017, 11, 5), datetime.date(2017, 11, 12))
004 ND | 62 ('enterprise', 'E115CH', 'Dd Elmwood Park') ('14.15', '41.926269', '-87.821333')
(datetime.date(2017, 11, 6), datetime.date(2017, 11, 13))
(datetime.date(2017, 11, 7), datetime.date(2017, 11, 14))
(datetime.date(2017, 11, 8), datetime.date(2017, 11, 15))
(datetime.date(2017, 11, 9), datetime.date(2017, 11, 16))
(datetime.date(2017, 11, 10), datetime.date(2017, 11, 17))
(datetime.date(2017, 11, 11), datetime.date(2017, 11, 18))
NO MATCH
(datetime.date(2017, 11, 12), datetime.date(2017, 11, 19))
011 ND | 62 ('enterprise', 'E115CH', 'Dd Elmwood Park') ('14.15', '41.926269', '-87.821333')
(da

In [10]:
car1 = 'Economy Car'
car2 = 'Compact Car'
car3 = 'Intermediate Car'
car4 = 'Intermediate SUV'
desiredcar = car2
#desiredsort = 'time'
desiredsort = desiredcar
desiredbrand = 'enterprise'

# also, can optimize into some single query...buuuut it works as is
rvals = np.array([x for x in results.values() if x is not None and desiredcar in x and desiredsort in x])
orderpreferred = np.array([x[desiredsort] for x in rvals]).argsort()
orderprice = np.array([x[desiredcar] for x in rvals]).argsort()
order = [x for x in orderpreferred if x in orderprice[:60]]

In [11]:
orderedresults = rvals[order]
print(desiredcar,car2,car3)
for k in orderedresults:
    #print(k['num'])
    if 'time' in k:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),k['time']//60,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))
    else:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),0,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))

Compact Car Compact Car Intermediate Car
11/13/2017-11/20/2017 | 14.15 |   0 | 128.81 | 128.81 | 140.17 | enterprise | Dd Elmwood Park
11/10/2017-11/17/2017 | 14.15 |   0 | 128.81 | 128.81 | 140.17 | enterprise | Dd Elmwood Park
11/03/2017-11/10/2017 | 14.15 |   0 | 129.97 | 129.97 | 140.96 | enterprise | Dd Elmwood Park
11/02/2017-11/09/2017 | 14.15 |   0 | 131.96 | 131.96 | 149.95 | enterprise | Dd Elmwood Park
11/06/2017-11/13/2017 | 14.15 |   0 | 149.97 | 149.97 | 159.96 | enterprise | Dd Elmwood Park
11/07/2017-11/14/2017 | 14.15 |   0 | 149.97 | 149.97 | 159.96 | enterprise | Dd Elmwood Park
11/09/2017-11/16/2017 | 14.15 |   0 | 149.97 | 149.97 | 167.95 | enterprise | Dd Elmwood Park
11/08/2017-11/15/2017 | 14.15 |   0 | 149.97 | 149.97 | 159.96 | enterprise | Dd Elmwood Park
